In [1]:
import time
import pandas as pd
import numpy as np
import sklearn.metrics
from datetime import date
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import re
import requests
from bs4 import BeautifulSoup

from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [109]:
start_time = time.time()
teams_list = ['avalanche','wild','blues','stars','predators','blackhawks','coyotes','jets', # Central
              'sharks','kraken','canucks','oilers','ducks','flames','golden_knights','kings',# Pacific  
              'sabres','bruins','maple_leafs','panthers','red_wings','senators','lightning','canadiens',# Atlantic  
              'blue_jackets','penguins','devils','capitals','rangers','flyers','islanders','hurricanes'# Metro 
             ]

roster_df = pd.DataFrame(
        {
            'team': 'xx',
            'player_name': 'xx',
            'salary':'xx',
            'contract_status': 'xx',
            'roster_status':'xx'
        },index=[0]
    ) 

for team in teams_list:
    print(team)
    url = 'https://www.capfriendly.com/teams/{0}'.format(team)
    req = requests.get(url).text
    soup = BeautifulSoup(req,'lxml')
    body = soup.find('tbody')
    team_table = soup.find('table',id='team')

    num_players = len(team_table.find_all('tr'))

    rostered = 'Rostered'



    for p in range(0,num_players):
        int_time = time.time()
        players = team_table.find_all('tr')[p]
        player_name_pre_stripped = players.find_all('td')[0].text
        try:
            if player_name_pre_stripped[-1] == '"':
                player_name = player_name_pre_stripped[:-4]
            else: 
                player_name = player_name_pre_stripped
        except:
            player_name = player_name_pre_stripped
        try:
            salary_detail = players.find_all('td',class_='center',style='position:rel')[0]
            salary = salary_detail.find_all()[0].text
            if salary.strip()[0] == '$':
                salary = int(salary[1:].replace(",",""))
                contract_status = 'contracted'
            else: 
                contract_status = salary
                salary = 0
            temp = pd.DataFrame(
                    {
                        'team': team,
                        'player_name': player_name,
                        'salary': salary,
                        'contract_status':contract_status,
                        'roster_status': rostered
                    },index=[0]
                ) 
            roster_df = roster_df.append(temp)
        except:       
            if str(player_name.strip()) == 'ROSTER SIZE':
                rostered = 'Not Rostered'
            else: 
                pass
    if team in ['jets','kings','canadiens','hurricanes']:
        print('Total time: {0} seconds'.format(np.round(time.time() - start_time)))    
            
roster_df = roster_df.loc[roster_df['player_name']!= 'xx'].reset_index()
roster_df.groupby('team').count()


avalanche
wild
blues
stars
predators
blackhawks
coyotes
jets
Total time: 5.0 seconds
sharks
kraken
canucks
oilers
ducks
flames
golden_knights
kings
Total time: 10.0 seconds
sabres
bruins
maple_leafs
panthers
red_wings
senators
lightning
canadiens
Total time: 15.0 seconds
blue_jackets
penguins
devils
capitals
rangers
flyers
islanders
hurricanes
Total time: 21.0 seconds


,index,player_name,salary,contract_status,roster_status
team,,,,,
avalanche,52,52,52,52,52
blackhawks,46,46,46,46,46
blue_jackets,50,50,50,50,50
blues,49,49,49,49,49
bruins,49,49,49,49,49
canadiens,54,54,54,54,54
canucks,52,52,52,52,52
capitals,48,48,48,48,48
coyotes,51,51,51,51,51


In [110]:
roster_df.head(10)

,index,team,player_name,salary,contract_status,roster_status
0,0,avalanche,"Rantanen, Mikko",9250000,contracted,Rostered
1,0,avalanche,"Landeskog, Gabriel",7000000,contracted,Rostered
2,0,avalanche,"MacKinnon, Nathan",6300000,contracted,Rostered
3,0,avalanche,"Nichushkin, Valeri",6125000,contracted,Rostered
4,0,avalanche,"Compher, J.T.",3500000,contracted,Rostered
5,0,avalanche,"Cogliano, Andrew",1250000,contracted,Rostered
6,0,avalanche,"O'Connor, Logan",1050000,contracted,Rostered
7,0,avalanche,"Newhook, Alex",908333,contracted,Rostered
8,0,avalanche,"Sedlak, Lukas",800000,contracted,Rostered
9,0,avalanche,"Lehkonen, Artturi",0,RFA,Rostered


In [112]:
roster_df.to_csv('~/desktop/python/nhl_player_salaries.csv',header=None,index=False)